In [4]:
probability_dict = {} #placeholder for CPT table that we will calculate from another file


In [5]:
import numpy as np

class BlackjackAI:
    def __init__(self, probabilities):
        """
        Initializes the AI agent with probabilities for decision-making.
        :param probabilities: Dictionary of probability estimates (from dataset analysis)
        """
        self.probabilities = probabilities  # Precomputed probabilities from data

    def get_action(self, player_hand, dealer_card, bet_stage):
        """
        Determines the best action based on the player's hand, dealer's face-up card, and betting stage.
        :param player_hand: List of player's cards
        :param dealer_card: The dealer's face-up card
        :param bet_stage: Which bet stage the player is in (1, 2, or 3)
        :return: Chosen action ("Hit", "Stand", "Split", "Bet", or "Swap")
        """
        hand_sum = sum(player_hand)
        num_cards = len(player_hand)
        
        # **1. HIT Decision**: If probability of busting is low
        bust_probability = self.get_bust_probability(hand_sum)
        if hand_sum < 17 and bust_probability < 0.3:
            return "Hit"
        
        # **2. STAND Decision**: If bust probability is high or hand sum is strong
        if hand_sum >= 17 or bust_probability >= 0.5:
            return "Stand"
        
        # **3. SPLIT Decision**: If splitting is beneficial
        if player_hand[0] == player_hand[1] and self.get_split_value(player_hand, dealer_card) > 0:
            return "Split"
        
        # **4. BET Decision**: Scale betting based on hand strength
        if bet_stage < 3:  # Can only bet up to 3 times
            return "Bet"
        
        # **5. SWAP Decision**: If swapping improves hand value
        if self.should_swap(player_hand, dealer_card):
            return "Swap"
        
        return "Stand"  # Default conservative move

    def get_bust_probability(self, hand_sum):
        """Returns probability of busting if hitting."""
        return self.probabilities.get(f"bust_prob_{hand_sum}", 0.5)  # Default 50% if not precomputed

    def get_split_value(self, player_hand, dealer_card):
        """Returns expected value increase of splitting."""
        return self.probabilities.get(f"split_value_{player_hand[0]}_{dealer_card}", 0)

    def should_swap(self, player_hand, dealer_card):
        """Determines whether swapping is beneficial."""
        swap_value = self.probabilities.get(f"swap_benefit_{player_hand[0]}_{player_hand[1]}_{dealer_card}", 0)
        return swap_value > 0


In [6]:
# arbitrary value to show agent function
probability_data = {
    "bust_prob_15": 0.25, "bust_prob_16": 0.35, "bust_prob_17": 0.50,
    "split_value_8_6": 1, "swap_benefit_5_10_6": 0.2
}
agent = BlackjackAI(probabilities=probability_data)
player_hand = [10, 6]
dealer_card = 5  
bet_stage = 2  
print("Agent Decision:", agent.get_action(player_hand, dealer_card, bet_stage))

Agent Decision: Bet


In [7]:
import random

# Simulate 1,000 rounds of blackjack
def simulate_blackjack(agent, num_rounds=1000):
    results = {"win": 0, "loss": 0}
    
    for _ in range(num_rounds):
        # Generate random starting hand
        player_hand = [random.randint(2, 11), random.randint(2, 11)]
        dealer_card = random.randint(2, 11)
        bet_stage = random.choice([1, 2, 3])
        
        # Get AI decision
        action = agent.get_action(player_hand, dealer_card, bet_stage)
        
        # Simulate game outcome (simplified logic)
        if action == "Hit":
            if sum(player_hand) + random.randint(2, 11) > 21:  # Random hit
                results["loss"] += 1  # Busted
            else:
                results["win"] += 1
        elif action in ["Stand", "Split", "Swap"]:
            results["win"] += 1  # Conservative play gives better odds
        else:
            results["loss"] += 1  # Risky bets may lose
            
    return results

# Run the simulation
simulation_results = simulate_blackjack(agent)
print("Simulation Results:", simulation_results)


Simulation Results: {'win': 903, 'loss': 97}
